# Creación de DataFrames
Crearemos objetos `DataFrame` a partir de otras estructuras de datos en Python, leyendo un fichero CSV, y consultando una base de datos.

## Acerca de los datos
En este cuaderno trabajaremos con datos de terremotos del 18 de septiembre de 2018 al 13 de octubre de 2018 (obtenidos del US Geological Survey (USGS) usando la [USGS API](https://earthquake.usgs.gov/fdsnws/event/1/))

## Importaciones

In [1]:
import datetime as dt
import numpy as np
import pandas as pd

## Creación de un objeto `Series`

In [2]:
np.random.seed(0) # fijar la semilla para que el resultado sea reproducible
pd.Series(np.random.rand(5), name='random')

0    0.548814
1    0.715189
2    0.602763
3    0.544883
4    0.423655
Name: random, dtype: float64

## Crear un objeto `DataFrame` a partir de un objeto `Series`
Utilice el método `to_frame()`:

> to_frame() es un método de Pandas series, que convierte en dataframe.

In [3]:
pd.Series(np.linspace(0, 10, num=5)).to_frame()

,0
0,0.0
1,2.5
2,5.0
3,7.5
4,10.0


## Creación de un `DataFrame` a partir de Estructuras de Datos Python
### A partir de un diccionario de estructuras tipo lista
Los valores del diccionario pueden ser listas, matrices NumPy, etc. siempre que tengan longitud (los generadores no tienen longitud por lo que no podemos usarlos aquí):

In [4]:
np.random.seed(0) # fijar la semilla para que el resultado sea reproducible
pd.DataFrame(
    {
        'random': np.random.rand(5),
        'texto': ['caliente', 'tibio', 'fresco', 'frio', None],
        'verdad': [np.random.choice([True, False]) for _ in range(5)]
    }, 
    index=pd.date_range(
        end=dt.date(2019, 4, 21),
        freq='1D', # Frecuencia diaria
        periods=5, 
        name='date'
    )
)

,random,texto,verdad
date,,,
2019-04-17,0.548814,caliente,False
2019-04-18,0.715189,tibio,True
2019-04-19,0.602763,fresco,True
2019-04-20,0.544883,frio,False
2019-04-21,0.423655,None,True


> date_range es un range normal pero va por fecha.

### De una lista de diccionarios

In [5]:
pd.DataFrame([
    {'mag': 5.2, 'place': 'California'},
    {'mag': 1.2, 'place': 'Alaska'},
    {'mag': 0.2, 'place': 'California'},
])

,mag,place
0,5.2,California
1,1.2,Alaska
2,0.2,California


### De una lista de tuplas

In [6]:
list_of_tuples = [(n, n**2, n**3) for n in range(5)]
list_of_tuples

[(0, 0, 0), (1, 1, 1), (2, 4, 8), (3, 9, 27), (4, 16, 64)]

In [7]:
pd.DataFrame(
    list_of_tuples, 
    columns=['n', 'n_cuadrado', 'n_cubo']
)

,n,n_cuadrado,n_cubo
0,0,0,0
1,1,1,1
2,2,4,8
3,3,9,27
4,4,16,64


### De un array NumPy

In [8]:
pd.DataFrame(
    np.array([
        [0, 0, 0],
        [1, 1, 1],
        [2, 4, 8],
        [3, 9, 27],
        [4, 16, 64]
    ]), columns=['n', 'n_cuadrado', 'n_cubo']
)

,n,n_cuadrado,n_cubo
0,0,0,0
1,1,1,1
2,2,4,8
3,3,9,27
4,4,16,64


## Creación de un objeto `DataFrame` a partir del contenido de un fichero CSV

### Buscando información sobre el fichero antes de leerlo
Antes de intentar leer un fichero, podemos usar la línea de comandos para ver información importante sobre el fichero que puede determinar cómo lo leemos. Podemos ejecutar código de línea de comandos desde Jupyter Notebooks (gracias a IPython) usando `!` antes del código.

#### Número de líneas (recuento de filas)
Por ejemplo, podemos averiguar cuántas líneas hay en el archivo utilizando la utilidad `wc` (recuento de palabras) y contando las líneas del archivo (`-l`). El fichero tiene 9.333 líneas:

In [9]:
!wc -l data/earthquakes.csv

    9333 data/earthquakes.csv


**Usuarios de Windows**: si lo anterior no te funciona (depende de tu configuración), utiliza esto en su lugar:

```python
!find /c /v "" data\earthquakes.csv
```



#### Tamaño del archivo
Podemos encontrar el tamaño del archivo usando `ls` para listar los archivos en el directorio `data`, y pasando las banderas `-lh` para incluir el tamaño del archivo en formato legible por humanos. Luego usamos `grep` para encontrar el fichero en cuestión. Ten en cuenta que `|` pasa el resultado de `ls` a `grep`. La utilidad `grep` se utiliza para encontrar elementos que coincidan con patrones.

Esto nos dice que el archivo tiene 3,4 MB:

In [10]:
!ls -lh data | grep earthquakes.csv

-rw-r--r--  1 ivanet  staff   3.4M Oct 27 20:37 earthquakes.csv



**Usuarios de Windows**: si lo anterior no te funciona (depende de tu configuración), utiliza esto en su lugar:

``python
!dir data | findstr "earthquakes.csv"
```

Incluso podemos capturar el resultado de un comando y utilizarlo en nuestro código Python:

In [11]:
files = !ls -lh data
[file for file in files if 'earthquake' in file]

['-rw-r--r--  1 ivanet  staff   3.4M Oct 27 20:37 earthquakes.csv']

**Usuarios de Windows**: si lo anterior no te funciona (depende de tu configuración), utiliza esto en su lugar:

```python
files = !dir data
[file for file in files if 'earthquake' in file]
```


#### Examinar algunas filas
Podemos utilizar `head` para examinar las `n` primeras filas del fichero. Con la bandera `-n`, podemos especificar cuántas. Esto nos muestra que la primera fila del fichero contiene cabeceras y que están separadas por comas (el hecho de que la extensión del fichero sea `.csv` no significa que contenga valores separados por comas):

In [12]:
!head -n 2 data/earthquakes.csv

alert,cdi,code,detail,dmin,felt,gap,ids,mag,magType,mmi,net,nst,place,rms,sig,sources,status,time,title,tsunami,type,types,tz,updated,url
,,37389218,https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ci37389218&format=geojson,0.008693,,85.0,",ci37389218,",1.35,ml,,ci,26.0,"9km NE of Aguanga, CA",0.19,28,",ci,",automatic,1539475168010,"M 1.4 - 9km NE of Aguanga, CA",0,earthquake,",geoserve,nearby-cities,origin,phase-data,",-480.0,1539475395144,https://earthquake.usgs.gov/earthquakes/eventpage/ci37389218


> La fila de arriba son las cabeceras, y a cada una le corresponde como valor lo de la fila de abajo, que está separado por comas.

**Usuarios de Windows**: si lo anterior no te funciona (depende de tu configuración), utiliza esto en su lugar:

```python
n = 2
with open('data/earthquakes.csv', 'r') as file:
    for _ in range(n):
        print(file.readline(), end='\r')
```


Igual que `head` da las filas de arriba, `tail` da las filas de abajo. Esto puede ayudarnos a comprobar que no hay datos extraños en la parte inferior del campo, como quizás algunos metadatos sobre los campos que en realidad no forman parte del conjunto de datos:

In [13]:
!tail -n 1 data/earthquakes.csv

,,38063935,https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ci38063935&format=geojson,0.01698,,39.0,",ci38063935,",0.66,ml,,ci,24.0,"9km NE of Aguanga, CA",0.1,7,",ci,",reviewed,1537228864470,"M 0.7 - 9km NE of Aguanga, CA",0,earthquake,",focal-mechanism,geoserve,nearby-cities,origin,phase-data,scitech-link,",-480.0,1537305830770,https://earthquake.usgs.gov/earthquakes/eventpage/ci38063935


**Usuarios de Windows**: si lo anterior no te funciona (depende de tu configuración), utiliza esto en su lugar:

```python
import os

with open('data/earthquakes.csv', 'rb') as file:
    file.seek(0, os.SEEK_END)
    while file.read(1) != b'\n':
        file.seek(-2, os.SEEK_CUR)
    print(file.readline().decode())
```

*Nota Para inspeccionar más de una fila desde el final del fichero, tendrás que usar esto en su lugar, lo que requiere leer todo el fichero:

```python
n = 2
with open('data/earthquakes.csv', 'r') as file:
    print('\r'.join(file.readlines()[-n:]))
```

#### Recuento de columnas
Podemos utilizar `awk` para encontrar el recuento de columnas. Se trata de una utilidad para escanear y procesar patrones. La bandera `-F` nos permite especificar el delimitador (coma, en este caso). Luego especificamos qué hacer para cada registro del fichero. Elegimos imprimir `NF`, que es una variable predefinida cuyo valor es el número de campos del registro actual. Aquí, decimos `exit` para que imprimamos el número de campos (columnas, aquí) en la primera fila del fichero, y luego nos detenemos.

Esto nos dice que tenemos 26 columnas de datos:

In [14]:
!awk -F',' '{print NF; exit}' data/earthquakes.csv

26



**Usuarios de Windows**: si lo anterior o lo siguiente no te funciona (depende de tu configuración), utiliza esto en su lugar:

```python
with open('data/earthquakes.csv', 'r') as file:
    print(len(file.readline().split(',')))
```


Como sabemos que la primera línea del fichero tiene cabeceras, y el fichero está separado por comas, también podemos contar las columnas usando `head` para obtener las cabeceras y analizarlas en Python:

In [17]:
headers = !head -n 1 data/earthquakes.csv
len(headers[0].split(','))

26

> 26 son el número de columnas que hay (headers).

**Usuarios de Windows**: si tiene que utilizar las alternativas anteriores, considere probar [Cygwin](https://www.cygwin.com) o [Windows Subsystem for Linux (WSL)](https://docs.microsoft.com/en-us/windows/wsl/about).

### Lectura del fichero
Nuestro fichero es de pequeño tamaño, tiene cabeceras en la primera fila, y está separado por comas, por lo que no necesitamos proporcionar ningún argumento adicional para leer el fichero con `pd.read_csv()`, pero asegúrate de revisar la [documentación](http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) para posibles argumentos:

In [22]:
df = pd.read_csv('data/earthquakes.csv')
df
df.head()

,alert,cdi,code,detail,dmin,felt,gap,ids,mag,magType,...,sources,status,time,title,tsunami,type,types,tz,updated,url
0,NaN,NaN,37389218,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.008693,NaN,85.0,",ci37389218,",1.35,ml,...,",ci,",automatic,1539475168010,"M 1.4 - 9km NE of Aguanga, CA",0,earthquake,",geoserve,nearby-cities,origin,phase-data,",-480.0,1539475395144,https://earthquake.usgs.gov/earthquakes/eventp...
1,NaN,NaN,37389202,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.020030,NaN,79.0,",ci37389202,",1.29,ml,...,",ci,",automatic,1539475129610,"M 1.3 - 9km NE of Aguanga, CA",0,earthquake,",geoserve,nearby-cities,origin,phase-data,",-480.0,1539475253925,https://earthquake.usgs.gov/earthquakes/eventp...
2,NaN,4.4,37389194,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.021370,28.0,21.0,",ci37389194,",3.42,ml,...,",ci,",automatic,1539475062610,"M 3.4 - 8km NE of Aguanga, CA",0,earthquake,",dyfi,focal-mechanism,geoserve,nearby-cities,o...",-480.0,1539536756176,https://earthquake.usgs.gov/earthquakes/eventp...
3,NaN,NaN,37389186,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.026180,NaN,39.0,",ci37389186,",0.44,ml,...,",ci,",automatic,1539474978070,"M 0.4 - 9km NE of Aguanga, CA",0,earthquake,",geoserve,nearby-cities,origin,phase-data,",-480.0,1539475196167,https://earthquake.usgs.gov/earthquakes/eventp...
4,NaN,NaN,73096941,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.077990,NaN,192.0,",nc73096941,",2.16,md,...,",nc,",automatic,1539474716050,"M 2.2 - 10km NW of Avenal, CA",0,earthquake,",geoserve,nearby-cities,origin,phase-data,scit...",-480.0,1539477547926,https://earthquake.usgs.gov/earthquakes/eventp...


Pandas es normalmente muy bueno en averiguar qué opciones utilizar basándose en los datos de entrada, por lo que a menudo no necesitaremos añadir argumentos a la llamada; sin embargo, hay muchas opciones disponibles si las necesitamos, algunas de las cuales incluyen las siguientes:

| Parámetro | Propósito |
| --- | --- |
| `sep` | Especifica el delimitador |
| `header` | Número de fila donde se encuentran los nombres de las columnas; la opción por defecto hace que `pandas` deduzca si están presentes |
| `names` | Lista de nombres de columnas para utilizar como cabecera |
| `index_col` | Columna que se utilizará como índice |
| `usecols` | Especifica qué columnas leer |
| `dtype` | Especifica los tipos de datos de las columnas | 
| `converters` | Especifica funciones para convertir datos en determinadas columnas |
| `skiprows` | Filas a saltar |
| `nrows` | Número de filas a leer a la vez (combinar con `skiprows` para leer un fichero bit a bit) |
| `parse_dates` | Convertir automáticamente las columnas que contienen fechas en objetos datetime |
| `chunksize` | Para leer el archivo en trozos |
| `compression` | Para leer archivos comprimidos sin extraerlos previamente |
| `encoding` | Especifica la codificación del archivo |

## Escribir un objeto `DataFrame` en un fichero CSV
Ten en cuenta que el índice de `df` son sólo números de fila, por lo que no queremos conservarlo. Por lo tanto, pasamos `index=False` al método `to_csv()`:

In [23]:
df.to_csv('output.csv', index=False)
# Se crea el fichero output.csv en la carpeta ch_02. 
# Podemos elegir la ruta donde queremos el archivo, por ejemplo: 'data/output.csv'

## Escribir un objeto `DataFrame` en una base de datos
Fíjate en el parámetro `if_exists`. Por defecto, te dará un error si intentas escribir una tabla que ya existe. Aquí, no nos importa si se sobrescribe. Por último, si estamos interesados en añadir nuevas filas, le damos el valor `'append'`.

In [26]:
!pip3 list

Package                   Version
------------------------- ------------
anyio                     4.0.0
appnope                   0.1.3
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asttokens                 2.4.0
async-lru                 2.0.4
attrs                     23.1.0
Babel                     2.13.1
backcall                  0.2.0
beautifulsoup4            4.12.2
bleach                    6.1.0
certifi                   2023.7.22
cffi                      1.16.0
charset-normalizer        3.3.1
comm                      0.1.4
contourpy                 1.1.1
cycler                    0.12.1
debugpy                   1.8.0
decorator                 5.1.1
defusedxml                0.7.1
executing                 2.0.0
fastjsonschema            2.18.1
fonttools                 4.43.1
fqdn                      1.5.1
graphviz                  0.20.1
greenlet                  3.0.1
idna                      3.4
imbalanced-learn   

In [24]:
from sqlalchemy import create_engine
import pandas as pd

engine = create_engine('sqlite:///data/quakes.db') # .db = database
# Si el archivo quakes.db no existiera, lo crea.
# Si existe, sobreescribe lo que le estamos diciendo.

pd.read_csv('data/tsunamis.csv').to_sql(
    'tsunamis', engine, index=False, if_exists='replace'
)


61

## Crear un objeto `DataFrame` consultando una base de datos
Usando una base de datos SQLAlchemy

In [27]:
from sqlalchemy import create_engine
import pandas as pd

engine = create_engine('sqlite:///data/quakes.db')

tsunamis = pd.read_sql('SELECT * FROM tsunamis', engine)

tsunamis.head()


,alert,type,title,place,magType,mag,time
0,None,earthquake,"M 5.0 - 165km NNW of Flying Fish Cove, Christm...","165km NNW of Flying Fish Cove, Christmas Island",mww,5.0,1539459504090
1,green,earthquake,"M 6.7 - 262km NW of Ozernovskiy, Russia","262km NW of Ozernovskiy, Russia",mww,6.7,1539429023560
2,green,earthquake,"M 5.6 - 128km SE of Kimbe, Papua New Guinea","128km SE of Kimbe, Papua New Guinea",mww,5.6,1539312723620
3,green,earthquake,"M 6.5 - 148km S of Severo-Kuril'sk, Russia","148km S of Severo-Kuril'sk, Russia",mww,6.5,1539213362130
4,green,earthquake,"M 6.2 - 94km SW of Kokopo, Papua New Guinea","94km SW of Kokopo, Papua New Guinea",mww,6.2,1539208835130


<hr>
<div>
    <a href="./1-pandas_data_estructura.ipynb">
        <button style="float: left;">&#8592; Notebook Anterior</button>
    </a>
    <a href="./3-creando_dataframes_con_api_requests.ipynb">
        <button style="float: right;">Siguiente Notebook &#8594;</button>
    </a>
</div>
<br>
<hr>